In [1]:
# Update to transformers 2.8.0
!pip install -q transformers --upgrade
!pip show transformers

Name: transformers
Version: 2.8.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.6/site-packages
Requires: numpy, dataclasses, sentencepiece, boto3, tokenizers, filelock, requests, sacremoses, regex, tqdm
Required-by: 


In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, TFAutoModel, TFElectraModel, ElectraTokenizer
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

In [3]:
tqdm.pandas()

## Helper functions

In [4]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512, enable_padding=False):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    
    ---
    
    Inputs:
        tokenizer: the `fast_tokenizer` that we imported from the tokenizers library
    """
    tokenizer.enable_truncation(max_length=maxlen)
    if enable_padding:
        tokenizer.enable_padding(max_length=maxlen)
    
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [5]:
def combine_qa_ids(q_ids, a_ids, tokenizer, maxlen=512):
    """
    Given two arrays of IDs (questions and answers) created by
    `fast_encode`, we combine and pad them.
    Inputs:
        tokenizer: The original tokenizer (not the fast_tokenizer)
    """
    combined_ids = []

    for i in tqdm(range(q_ids.shape[0])):
        ids = []
        ids.append(tokenizer.cls_token_id)
        ids.extend(q_ids[i])
        ids.append(tokenizer.sep_token_id)
        ids.extend(a_ids[i])
        ids.append(tokenizer.sep_token_id)
        ids.extend([tokenizer.pad_token_id] * (maxlen - len(ids)))

        combined_ids.append(ids)
    
    return np.array(combined_ids)

In [6]:
def encode_qa(questions, answers, tokenizer, chunk_size=256, maxlen=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(questions), chunk_size)):
        q_chunk = questions[i:i+chunk_size].tolist()
        a_chunk = answers[i:i+chunk_size].tolist()
        text_chunk = list(zip(q_chunk, a_chunk))
        
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [7]:
def truncate_text(text, tokenizer, chunk_size=256, maxlen=256):
    """
    Ensure that the text does not have more than maxlen tokens
    """
    tokenizer.enable_truncation(max_length=maxlen)
    all_norm_str = []
    
    for i in tqdm(range(0, len(text), chunk_size)):
        chunk = text[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(chunk)
        all_norm_str.extend([str(enc.normalized_str) for enc in encs])
    
    return all_norm_str

In [8]:
def build_model(transformer, max_len=None):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_ids = L.Input(shape=(max_len, ), dtype=tf.int32)
    
    x = transformer(input_ids)[0]
    x = x[:, 0, :]
    x = L.Dense(1, activation='sigmoid', name='sigmoid')(x)
    
    # BUILD AND COMPILE MODEL
    model = Model(inputs=input_ids, outputs=x)
    model.compile(
        loss='binary_crossentropy', 
        metrics=['accuracy'], 
        optimizer=Adam(lr=1e-5)
    )
    
    return model

In [9]:
def save_model(model, sigmoid_dir='transformer', transformer_dir='transformer'):
    """
    Special function to load a keras model that uses a transformer layer
    """
    os.makedirs(transformer_dir, exist_ok=True)
    os.makedirs(sigmoid_dir, exist_ok=True)
    
    transformer = model.layers[1]
    transformer.save_pretrained(transformer_dir)
    
    sigmoid_path = os.path.join(sigmoid_dir,'sigmoid.pickle')
    sigmoid = model.get_layer('sigmoid').get_weights()
    pickle.dump(sigmoid, open(sigmoid_path, 'wb'))

In [10]:
    
def load_model(sigmoid_dir='transformer', transformer_dir='transformer', 
               architecture="electra", max_len=None):
    """
    Special function to load a keras model that uses a transformer layer
    """
    sigmoid_path = os.path.join(sigmoid_dir,'sigmoid.pickle')
    
    if architecture == 'electra':
        transformer = TFElectraModel.from_pretrained(transformer_dir)
    else:
        transformer = TFAutoModel.from_pretrained(transformer_dir)
    model = build_model(transformer, max_len=max_len)
    
    sigmoid = pickle.load(open(sigmoid_path, 'rb'))
    model.get_layer('sigmoid').set_weights(sigmoid)
    
    return model

## TPU Configs

In [11]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [12]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 8
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 512
MODEL = 'google/electra-small-discriminator'

## Load data

In [13]:
df = pd.concat([
    pd.read_csv(f'/kaggle/input/stackexchange-qa-pairs/pre_covid/{group}.csv')
    for group in ['expert', 'biomedical', 'general']
]).reset_index(drop=True)

df.head()

,Unnamed: 0,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,site,group
0,0,398847,How to show UML component load/discovery and r...,In a UML component diagram I want to show two ...,398923,Your first diagram is not correct. Arrow A wou...,Accepted,"This way, the dates can easily be sorted as st...",Random,programmers,expert
1,1,398849,Queue management project - Recommendations,I would like to develop a Patient Queue manage...,398889,Design issue\n\nIn reality you have two very d...,Accepted,If the capabilities of your new ISA are simila...,Random,programmers,expert
2,2,398856,Why is it bad to map between a Model and a Vie...,"In a recent Pull Request (PR) of mine, a colle...",398858,\n It isn't how extension methods are suppose...,Accepted,You can do all the basic validation on client-...,Random,programmers,expert
3,3,398857,Managing sprints for multiple releases of the ...,How do you plan a sprint for the same product ...,398888,Sprints are different than releases. Although ...,Accepted,Create a repository to wrap whatever stores yo...,Random,programmers,expert
4,4,398863,Does references between entities of the same a...,Consider an application being implemented foll...,398866,"To your question: ""Does the usage of these ref...",Accepted,I would do the second option but allow the use...,Random,programmers,expert


In [14]:
questions = df.title + "[SEP]" + df.question

## Bert tokenizer

In [15]:
# First load the real tokenizer
tokenizer = ElectraTokenizer.from_pretrained(MODEL)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True, add_special_tokens=False)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, add_special_tokens=False, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)

## Convert text to matrices

Caveat: Since a lot of the questions on stackexchange goes over 256, characters, we end up truncating a large part (if not all) of the answers. Thus, we need to "pre" truncate them by separately encode the questions and answers, and use a functions to combine them again.

Note: Here we are not actually encoding it, instead we load the encoded q&a pairs from another notebook, in order to limit memory consumption.

In [16]:
# q_ids = fast_encode(questions.values, fast_tokenizer, maxlen=MAX_LEN//2 - 2)
# a_ids = fast_encode(df.answer.values, fast_tokenizer, maxlen=MAX_LEN//2 - 2)
# wa_ids = fast_encode(df.wrong_answer.values, fast_tokenizer, maxlen=MAX_LEN//2 - 2)

# correct_ids = combine_qa_ids(q_ids, a_ids, tokenizer, maxlen=MAX_LEN)
# wrong_ids = combine_qa_ids(q_ids, wa_ids, tokenizer, maxlen=MAX_LEN)

In [17]:
correct_ids = np.load('/kaggle/input/stackexchange-encode-for-electra/correct_ids.npy')
wrong_ids = np.load('/kaggle/input/stackexchange-encode-for-electra/wrong_ids.npy')

In [18]:
input_ids = np.concatenate([correct_ids, wrong_ids])

labels = np.concatenate([
    np.ones(correct_ids.shape[0]),
    np.zeros(wrong_ids.shape[0])
]).astype(np.int32)

## Train test split

In [19]:
train_idx, test_idx = train_test_split(
    np.arange(input_ids.shape[0]), 
    test_size=0.3, 
    random_state=0
)

valid_idx, test_idx = train_test_split(
    test_idx, 
    test_size=0.5, 
    random_state=1
)

In [20]:
train_ids = input_ids[train_idx]
valid_ids = input_ids[valid_idx]
test_ids = input_ids[test_idx]

train_labels = labels[train_idx]
valid_labels = labels[valid_idx]
test_labels = labels[test_idx]

## Build datasets objects

In [21]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_ids, train_labels))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_ids, valid_labels))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_ids)
    .batch(BATCH_SIZE)
)

## Modeling

In [22]:
%%time
with strategy.scope():
    transformer_layer = TFElectraModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
tf_electra_model (TFElectraM ((None, 512, 256),)       13483008  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 256)]             0         
_________________________________________________________________
sigmoid (Dense)              (None, 1)                 257       
Total params: 13,483,265
Trainable params: 13,483,265
Non-trainable params: 0
_________________________________________________________________
CPU times: user 12.4 s, sys: 2.56 s, total: 14.9 s
Wall time: 17.7 s


### Train model

In [23]:
n_steps = train_labels.shape[0] // BATCH_SIZE

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 3209 steps, validate for 688 steps
Epoch 1/8
3209/3209 [==============================] - 573s 179ms/step - loss: 0.3794 - accuracy: 0.8315 - val_loss: 0.3082 - val_accuracy: 0.8763
Epoch 2/8
3209/3209 [==============================] - 481s 150ms/step - loss: 0.2834 - accuracy: 0.8858 - val_loss: 0.2920 - val_accuracy: 0.8825
Epoch 4/8
3209/3209 [==============================] - 478s 149ms/step - loss: 0.2710 - accuracy: 0.8909 - val_loss: 0.2844 - val_accuracy: 0.8868
Epoch 5/8
3209/3209 [==============================] - 477s 149ms/step - loss: 0.2596 - accuracy: 0.8957 - val_loss: 0.2822 - val_accuracy: 0.8866
Epoch 6/8
3209/3209 [==============================] - 483s 150ms/step - loss: 0.2322 - accuracy: 0.9071 - val_loss: 0.2898 - val_accuracy: 0.8888


In [24]:
save_model(model)

In [25]:
hist_df = pd.DataFrame(train_history.history)
hist_df.to_csv('train_history.csv')
hist_df

,loss,accuracy,val_loss,val_accuracy
0,0.379429,0.831492,0.308205,0.876306
1,0.301452,0.877814,0.310645,0.879282
2,0.283414,0.885783,0.291989,0.882508
3,0.271009,0.890932,0.284387,0.886779
4,0.259630,0.895664,0.282210,0.886608
5,0.250051,0.899596,0.285317,0.887824
6,0.240942,0.903509,0.287232,0.887426
7,0.232235,0.907131,0.289811,0.888755


## Eval

In [26]:
with strategy.scope():
    model = load_model(max_len=MAX_LEN)

In [27]:
y_score = model.predict(test_dataset, verbose=1).squeeze()
y_pred = y_score.round().astype(int)
print("AP:", average_precision_score(test_labels, y_score))
print("ROC AUC:", roc_auc_score(test_labels, y_score))
print(classification_report(test_labels, y_pred))

688/688 [==============================] - 67s 98ms/step
AP: 0.9336920747539842
ROC AUC: 0.9488429998168292
              precision    recall  f1-score   support

           0       0.94      0.84      0.88     44307
           1       0.85      0.94      0.89     43734

    accuracy                           0.89     88041
   macro avg       0.89      0.89      0.89     88041
weighted avg       0.89      0.89      0.89     88041

